# Plot the Near-Inertial Activity Forecast

To see how we got here, check the notebooks about [loading the data](./01_download_GFS_data.html), and about [running the slab-ocean model](./02_run_slab_model.html).

Details: https://willirath.github.io/nia-prediction-low-latitudes

## Parameters

In [ ]:
# parameters

GFS_zarr_store = "tmp_GFS.zarr"
slab_zarr_store = "tmp_slab.zarr"
buoy_file_name = "tmp_buoy_data"

dask_kwargs = {"n_workers": 1, "threads_per_worker": 2, "memory_limit": 6e9}

buoy_locations = {
    f"20.0N,{360.0-38.0}E": {"lat": 20.0, "lon": -38.0},
    f"15.0N,{360.0-38.0}E": {"lat": 15.0, "lon": -38.0},
    f"21.0N,{360.0-23.0}E": {"lat": 21.0, "lon": -23.0},
    f"12.0N,{360.0-23.0}E": {"lat": 12.0, "lon": -23.0},
    f"-6.0N,{360.0-10.0}E": {"lat": -6.0, "lon": -10.0},
    f"-10.0N,{360.0-10.0}E": {"lat": -10.0 ,"lon": -10.0},
}

lat_min, lat_max = -40, 40

## Tech preamble

Import modules and spin up Dask cluster.

In [ ]:
import numpy as np
import xarray as xr
from dask.distributed import Client
import hvplot.xarray, hvplot.pandas
import cmocean
import pandas as pd
import cartopy.crs as ccrs


import holoviews as hv

from functools import reduce
from operator import add

In [ ]:
client = Client(**dask_kwargs)
client

## Prepare buoy location plots

In [ ]:
buoy_df = pd.read_csv(f"{buoy_file_name}.csv")
buoy_df

In [ ]:
buoy_locations.update(
    {
        f"{lat}N,{lon}E": {"lat": lat, "lon": lon}
        for lat, lon in buoy_df.set_index(["lat", "lon"]).index.to_series().unique()
    }
)

In [ ]:
buoy_names = sorted(buoy_locations.keys(), key=lambda s: -float(s.split("N")[0]))
buoy_locations = {bn: buoy_locations[bn] for bn in buoy_names}

In [ ]:
display(buoy_locations)

## Load the data

In [ ]:
ds_GFS = xr.open_zarr(GFS_zarr_store)
ds_slab = xr.open_zarr(slab_zarr_store)

## Change SLP unig

In [ ]:
ds_GFS["SLP"] /= 100  # to hPa

## Restrict in latitude

In [ ]:
ds_GFS = ds_GFS.sel(lat=slice(lat_max, lat_min))
ds_slab = ds_slab.sel(lat=slice(lat_max, lat_min))

## Find start of forecast period

In [ ]:
start_of_forecast = (~ds_GFS["is_forecast"].astype(bool)).sum().compute().data
start_of_forecast = ds_GFS["time"].data[start_of_forecast]
print(start_of_forecast)

## Remove steady state

**FIXME:** This should have been done in the slab-model run?

In [ ]:
ds_slab["u_slab"] -= ds_slab["u_slab"].mean("time")
ds_slab["v_slab"] -= ds_slab["v_slab"].mean("time")
ds_slab["umag_slab"] = (ds_slab["u_slab"] ** 2 + ds_slab["v_slab"] ** 2) ** 0.5

## NIA timeseries for buoy locations

In [ ]:
plots = []

for name, location in buoy_locations.items():
    buoy_ds = ds_slab.sel(
        lat=location["lat"], lon=(360+location["lon"]) % 360, method="nearest"
    )
    buoy_ds["taux"] = ds_GFS["taux"].sel(
        lat=location["lat"], lon=(360+location["lon"]) % 360, method="nearest"
    )
    buoy_ds["tauy"] = ds_GFS["tauy"].sel(
        lat=location["lat"], lon=(360+location["lon"]) % 360, method="nearest"
    )
    
    plots.append(
        (
            buoy_ds["taux"].hvplot.line(label="taux", width=400, height=300)
            * buoy_ds["tauy"].hvplot.line(label="tauy")
        ).opts(legend_cols=True, title=name, legend_position='top')
        + (
            buoy_ds["u_slab"].hvplot.line(label="u", width=400, height=300)
            * buoy_ds["v_slab"].hvplot.line(label="v")
            * buoy_ds["umag_slab"].hvplot.line(label="umag")
            * hv.VLine(start_of_forecast)
        ).opts(legend_cols=True, title=name, legend_position='top')
    )

reduce(add, plots).cols(2)

## NIA max over forecast period

In [ ]:
slab_umag_max = ds_slab["umag_slab"].where(ds_GFS["is_forecast"]).max("time")
(
    slab_umag_max.hvplot(
        x="lon", y="lat", z="umag_slab",
        clim=(0, 2),
        cmap=cmocean.cm.speed,
        frame_width=800,
        hover=False,
        geo=True, coastline=True,
        crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
    )
    * pd.DataFrame.from_dict(buoy_locations).T.hvplot.points(
        y="lat", x="lon", geo=True, coastline=True,
        marker='circle',
        fill_color=None, line_color="black",
        line_width=1, size=60,
    )
).opts(title="Slab UMAG [m/s] max over forecast")

## Atmospheric conditions

In [ ]:
SLP_mean = ds_GFS["SLP"].mean("time")
SLP_anomaly = ds_GFS["SLP"] - SLP_mean

In [ ]:
plot_every = np.timedelta64(12, "h")
max_iter = ((SLP_anomaly.coords["time"].max("time") - start_of_forecast) / plot_every).item() // 1 + 1

plot_times = [
    (start_of_forecast + n * plot_every)
    for n in range(-6, int(max_iter))
]

plots = []

for plot_time in plot_times:
    try:
        plots.append(
            (
                SLP_anomaly.sel(time=plot_time, method="nearest").compute().hvplot(
                    clim=(-10, 10),
                    cmap=cmocean.cm.delta,
                    frame_width=800,
                    geo=True, coastline=True,
                    crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(),
                    hover=False
                )
                * pd.DataFrame.from_dict(buoy_locations).T.hvplot.points(
                    y="lat", x="lon", geo=True, coastline=True,
                    marker='circle',
                    fill_color=None, line_color="black",
                    line_width=1.5, size=60,
                )
            ).opts(title=f"SLP anomaly [hPa], {plot_time}")
        )    
    except Exception as e:
        print(f"for {plot_time} I got: {e}")
    
reduce(add, plots).cols(1)

---

In [ ]:
!echo "Finished: $(date -Ins)"

---
See https://github.com/willirath/nia-prediction-low-latitudes for details.